In [1]:
%matplotlib inline
from __future__ import division
import numpy as np
import os
import sys
import datetime
from subprocess import call
import subprocess
import glob
import djPyi2 as DJ
from djPyi2 import Common as CM
from djPyi2 import mpltools

import pandas as pd
import csv
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import copy 
import pybedtools as pbt
import ciepy
import cardipspy as cpy
import itertools
import tempfile
import six
import networkx as nx
import scipy.stats as stats
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
from mpl_toolkits.axes_grid1 import  make_axes_locatable
import datetime

from scipy.stats import mode
dy_name = 'hipstr_final_vcf'

private_out = os.path.join(DJ.root, 'private_output', dy_name)
if not os.path.exists(private_out):
    cpy.makedir(private_out)

# HipSTR VCF STRATEGY

    1)  Filter HipSTR Vcfs from iPSCORE / iPSCORE Hipsci REF/ HipSci 
        a) Filter iPSCORE_HipSci_REF to site NONREF_GT > 0 and CallRate > 80%
        b) Filter iPSCORE_Hipsci to site NONREF_GT > 0 and CallRate > 80%
        c) Filter iPSCORE to site NONREF_GT > 0 and CallRate > 80%
        d) Filter iPSCORE to sites not in ipscore_hipsci_ref - for merge with those
        
    2) Annotate Filtered VCFs- make sure we add suffixes to the majority of annotations in INFO so that bcftools can be used to easily merge and concat- as well as adjust for the headers
    
    Note- 1 and 2 are accomplished by a single script that reads/filters/ and annotates HipSTR vcfs at a specific set of IDs and modifies their existing INFO columns
    
    3) Concatenate iPSCORE unique with iPSCORE HipSci Ref with bcftools to create an iPSCORE_all file for merging
    4) Merge iPSCORE_all with HipSci using BCFtools
    
    5) Annotate on an additional set of annotations, namely for allele frequency/caller etc- using annotate_vcf_multi_info.py and also subset to samples that are needed and properly consented for sharing
    

In [489]:
DJ.convert_ipynb_to_script('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/HipSTR_processing/hipstr_generate_filtered_vcf.ipynb')

'/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/HipSTR_processing/hipstr_generate_filtered_vcf.py'

In [27]:
vcf = '/frazer01/projects/hipsci/pipeline/WGS/HipSTR/combined_results/hipstr_ipscore_ref_hipsci.filt.vcf'

In [8]:
filter_vcf_script = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/HipSTR_processing/hipstr_generate_filtered_vcf.py'

In [120]:
import datetime

In [121]:
    d = datetime.datetime.now()
    ts = d.strftime('%D- %H:%M')

In [124]:
l = [1, 2, 3, 5]

In [126]:
l.remove(5)

In [9]:
! python {filter_vcf_script}

usage: hipstr_generate_filtered_vcf.py [-h] -vcf <vcf_file> -id_fn <IDs to
                                       keep in the final output>
                                       [-suff <Suffix for INFO COL TAGS>] -o
                                       <out_dir>
hipstr_generate_filtered_vcf.py: error: argument -vcf/--vcf is required


In [106]:
def get_command(dir_master, vcf, suff = 'HipSci',ipscore = False):
    filter_vcf_script = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/HipSTR_processing/hipstr_generate_filtered_vcf.py'

    out_dir = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/{}'.format(dir_master)

    fn_id_ipscore_hipsci_ref = os.path.join(out_dir, 'IDs_retained.txt')

    fn = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/{}/hipstr_info_gt.tsv'.format(dir_master)

    info_gt = pd.read_table(fn)

    info_gt[info_gt.NREF > 0]['ID'].to_csv(fn_id_ipscore_hipsci_ref, header=False, index = False, sep = '\t')


    command = 'python {} -vcf {} -id_fn {} -suff {} -o {} -tag_hipsci'.format(filter_vcf_script, vcf, fn_id_ipscore_hipsci_ref, suff, out_dir)
    
    
#     if ipscore:
#         command = 'python {} -vcf {} -id_fn {} -suff {} -o {} -tag_hipsci'.format(filter_vcf_script, vcf, fn_id_ipscore_hipsci_ref, suff, out_dir)
    
        
    
    print command
    return command

In [357]:
tdf.NNONREF_DOSAGE_iPSCORE_HipSci_Ref.max()

272.0

In [108]:
vcf = '/frazer01/projects/hipsci/pipeline/WGS/HipSTR/combined_results/hipstr_ipscore_ref_hipsci.filt.vcf'

dir_master = 'ipscore_hipsci_ref_filt_sr_geno'

command = get_command(dir_master, vcf, suff = 'iPSCORE')

python /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/HipSTR_processing/hipstr_generate_filtered_vcf.py -vcf /frazer01/projects/hipsci/pipeline/WGS/HipSTR/combined_results/hipstr_ipscore_ref_hipsci.filt.vcf -id_fn /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/ipscore_hipsci_ref_filt_sr_geno/IDs_retained.txt -suff iPSCORE -o /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/ipscore_hipsci_ref_filt_sr_geno -tag_hipsci


In [110]:
vcf = '/frazer01/projects/hipsci/pipeline/WGS/HipSTR/combined_results/hipstr_hipsci.filt.vcf'

dir_master = 'hipsci_filt_geno'

command = get_command(dir_master, vcf, suff = 'HipSci')

python /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/HipSTR_processing/hipstr_generate_filtered_vcf.py -vcf /frazer01/projects/hipsci/pipeline/WGS/HipSTR/combined_results/hipstr_hipsci.filt.vcf -id_fn /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/hipsci_filt_geno/IDs_retained.txt -suff HipSci -o /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/hipsci_filt_geno -tag_hipsci


In [4]:
info_gt = pd.read_table('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/hipsci_filt_geno/hipstr_info_gt.tsv')

info_gt_ipscore  = pd.read_table('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/ipscore_filt_sr_geno/hipstr_info_gt.tsv')

info_gt_ih  = pd.read_table('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/ipscore_hipsci_ref_filt_sr_geno/hipstr_info_gt.tsv')

In [319]:
info_gt_ipscore  = pd.read_table('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/ipscore_filt_sr_geno/hipstr_info_gt.tsv')


In [320]:
ids_ipscore_all = info_gt_ipscore[info_gt_ipscore.NREF > 0].ID.tolist()

In [215]:
info_gt[info_gt.NREF > 0].shape

(660191, 25)

In [214]:
len(ids_ipscore_hipsci)

342231

In [31]:
ids_ipscore_hipsci = info_gt_ih[info_gt_ih.NREF > 0].ID.tolist()

ids_hipsci = info_gt[info_gt.NREF>0].ID.tolist()
ids_ipscore = info_gt_ipscore[info_gt_ipscore.NREF > 0]['ID'].tolist()


ids_diff = set(ids_ipscore_hipsci).difference(ids_hipsci)
unique_to_ipscore = set(ids_ipscore).difference(ids_hipsci + ids_ipscore_hipsci)

In [36]:
print 'number different between ipscore on hipsci ref and genotypes in hipsci samples'
len(ids_diff)

number different between ipscore on hipsci ref and genotypes in hipsci samples


8734

In [358]:
sample_info.groupby("CELL_TYPE").STUDY.value_counts()

CELL_TYPE   STUDY  
Blood       iPSCORE    254
            HLI          1
Fibroblast  HipSci     204
            iPSCORE     19
iPSC        HipSci     242
Name: STUDY, dtype: int64

Some sites were filtered eventually from hipsci because they failed missing rate, but they were still genotyped in ipscore, and didn't fail missing rate

In [25]:
info_gt_ih.index = info_gt_ih.ID
ids_ipscore = info_gt_ipscore[info_gt_ipscore.NREF > 0]['ID'].tolist()

In [40]:
len(ids_ipscore)

448325

In [41]:
unique_to_ipscore = set(ids_ipscore).difference(ids_hipsci + ids_ipscore_hipsci)

In [42]:
len(unique_to_ipscore)

95984

# Prep individual VCFs for merging

In [43]:
vcf = '/frazer01/projects/hipsci/pipeline/WGS/HipSTR/combined_results/hipstr_ipscore.filt.vcf'

fn_unique_to_ipscore = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/ipscore_filt_sr_geno/IDs_unique.txt'

out_dir = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/ipscore_filt_sr_geno'

suff = 'iPSCORE'

with open(fn_unique_to_ipscore,'w') as F:
    for l in unique_to_ipscore:
        F.write(l + '\n')

filter_vcf_script = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/HipSTR_processing/hipstr_generate_filtered_vcf.py'

command = 'python {} -vcf {} -id_fn {} -suff {} -o {} -tag_ipscore'.format(filter_vcf_script, vcf, fn_unique_to_ipscore, suff, out_dir)

! {command}

In [321]:
vcf = '/frazer01/projects/hipsci/pipeline/WGS/HipSTR/combined_results/hipstr_ipscore.filt.vcf'

fn_ipscore_all = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/ipscore_filt_sr_geno/IDs_all.txt'

out_dir = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf'
out_dir = os.path.join(out_dir, 'ipscore_all')
DJ.makedir(out_dir)


suff = 'iPSCORE'

with open(fn_ipscore_all,'w') as F:
    for l in ids_ipscore_all:
        F.write(l + '\n')

filter_vcf_script = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/HipSTR_processing/hipstr_generate_filtered_vcf.py'

command = 'python {} -vcf {} -id_fn {} -suff {} -o {} -tag_ipscore'.format(filter_vcf_script, vcf, fn_ipscore_all, suff, out_dir)
! {command}

In [323]:
len(ids_ipscore_all)

448325

In [322]:
command

'python /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/HipSTR_processing/hipstr_generate_filtered_vcf.py -vcf /frazer01/projects/hipsci/pipeline/WGS/HipSTR/combined_results/hipstr_ipscore.filt.vcf -id_fn /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/ipscore_filt_sr_geno/IDs_all.txt -suff iPSCORE -o /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/ipscore_all -tag_ipscore'

In [70]:
len(unique_to_ipscore)

95984

# Run BCFTOOLS MERGING

In [ ]:
! bgzip hipstr_output_annot.vcf
! tabix -p vcf hipstr_output_annot.vcf.gz

# prep vcfs for merging 
! bgzip ipscore_filt_sr_geno/hipstr_output_annot.vcf
! tabix -p vcf ipscore_filt_sr_geno/hipstr_output_annot.vcf.gz

! bgzip ipscore_hipsci_ref_filt_sr_geno/hipstr_output_annot.vcf
! tabix -p vcf ipscore_hipsci_ref_filt_sr_geno/hipstr_output_annot.vcf.gz

! bgzip hipsci_filt_geno/hipstr_output_annot.vcf
! tabix -p vcf hipsci_filt_geno/hipstr_output_annot.vcf.gz

! cd /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf
! bgzip hipstr_output_annot.vcf
! tabix -p vcf hipstr_output_annot.vcf.gz

In [204]:
output_dir = private_out + '/merged_vcf'
fn_ipscore_variants = os.path.join(output_dir, 'hipstr_ipscore_all.filt.vcf')
DJ.makedir(output_dir)

In [207]:
'bcftools concat -a ipscore_filt_sr_geno/hipstr_output_annot.vcf.gz  ipscore_hipsci_ref_filt_sr_geno/hipstr_output_annot.vcf.gz > {}'.format(fn_ipscore_variants)

'bcftools concat -a ipscore_filt_sr_geno/hipstr_output_annot.vcf.gz  ipscore_hipsci_ref_filt_sr_geno/hipstr_output_annot.vcf.gz > /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/merged_vcf/hipstr_ipscore_all.filt.vcf'

In [ ]:
! bcftools concat -a ipscore_filt_sr_geno/hipstr_output_annot.vcf.gz  ipscore_hipsci_ref_filt_sr_geno/hipstr_output_annot.vcf.gz > /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/merged_vcf/hipstr_ipscore_all.filt.vcf

In [208]:
fn_final_variants = os.path.join(output_dir, 'hipstr_i2QTL_merged.filt.vcf')

In [209]:
'bcftools merge -m id /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/merged_vcf/hipstr_ipscore_all.filt.vcf.gz hipsci_filt_geno/hipstr_output_annot.vcf.gz > {}'.format(fn_final_variants)

'bcftools merge -m id /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/merged_vcf/hipstr_ipscore_all.filt.vcf.gz hipsci_filt_geno/hipstr_output_annot.vcf.gz > /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/merged_vcf/hipstr_i2QTL_merged.filt.vcf'

In [ ]:
! bcftools merge -m id /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/merged_vcf/hipstr_ipscore_all.filt.vcf.gz hipsci_filt_geno/hipstr_output_annot.vcf.gz > /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/merged_vcf/hipstr_i2QTL_merged.filt.vcf

# Annotate AF Dosage and AF Geno to final vcf

We will make an info table to be annotated onto vcf using `annotate_vcf_multi_info.py`

In [142]:
def get_to_add(var_classes, categories, x, alternative_names = False):
    out = []
    for v in var_classes:
        for c in categories:
            col_name = '{}_{}'.format(c, v)
            info_entry = x[col_name]
            
            if alternative_names:
                alt_name = alternative_names[categories.index(c)]
                add = "{}={}".format(alt_name, info_entry)
            else:
                
                add = "{}={}".format(col_name, info_entry)
            out.append(add)
    return out

In [388]:
def info_column_merged(tdf):
    data = []
    for ind, x in tdf.iterrows():
        info = []
        in_both = x['in_both']
        categories = ['NNONREF_GT', 'NNONREF_FRAC_GT', 'NNONREF_DOSAGE', 'NNONREF_FRAC_DOSAGE']

        in_ipscore_hi = x['unique_to_ipscore_hi']
        in_hipsci = x['unique_to_hipsci']
        in_ipscore = x['unique_to_ipscore']

        combined_nref_gt = x['NNONREF_GT_COMBINED']
        combined_nref_frac_gt = x['NNONREF_FRAC_GT_COMBINED']

        combined_nref_d = x['NNONREF_DOSAGE_COMBINED']
        combined_nref_frac_d = x['NNONREF_FRAC_DOSAGE_COMBINED']


#         combined_categories = ['NNONREF_GT', 'NNONREF_FRAC_GT','NNONREF_DOSAGE',
#                                'NNONREF_FRAC_DOSAG' ]
        
#         combined_categories = ['NNONREF_GT_COMBINED', 'NNONREF_FRAC_GT_COMBINED','NNONREF_DOSAGE_COMBINED',
#                                'NNONREF_FRAC_DOSAGE_COMBINED' ]

        combined_categories_alt_names = ['NNONREF_GT_i2QTL', 'NNONREF_FRAC_GT_i2QTL','NNONREF_DOSAGE_i2QTL',
                               'NNONREF_FRAC_DOSAGE_i2QTL' ]



        if in_both:

            var_classes = ['HipSci', 'iPSCORE_HipSci_Ref']
            add = get_to_add(var_classes, categories, x)
            info += add

            var_classes = ['COMBINED']
            try:
                
                add = get_to_add(var_classes, categories, x, alternative_names=combined_categories_alt_names)
            except:
                x



        else:
            if in_ipscore_hi:
                var_classes = ['iPSCORE_HipSci_Ref']
                add = get_to_add(var_classes, categories, x)
                info += add

                var_classes = ['iPSCORE_HipSci_Ref']

                add = get_to_add(var_classes, categories, x, alternative_names=combined_categories_alt_names)
                info += add




            if in_hipsci:
                var_classes = ['HipSci']
                add = get_to_add(var_classes, categories, x)
                info += add

                var_classes = ['HipSci']

                add = get_to_add(var_classes, categories, x, alternative_names=combined_categories_alt_names)
                info += add



            if in_ipscore:
                var_classes = ['iPSCORE']
                add = get_to_add(var_classes, categories, x)
                info += add

                add = get_to_add(var_classes, categories, x, alternative_names=combined_categories_alt_names)
                info += add


        info.append("{}={}".format('CALLER', 'HipSTR'))

        info_str = ";".join(info)
        data.append([ind,info_str])
    info_df = pd.DataFrame(data, columns = ['ID', 'INFO'])
    return info_df

In [389]:
def info_column_hipstr_indiv(tdf, subtype='iPSCORE'):
    data = []
    for ind, x in tdf.iterrows():
        info = []
        in_both = x['in_both']
        categories = ['NNONREF_GT', 'NNONREF_FRAC_GT', 'NNONREF_DOSAGE', 'NNONREF_FRAC_DOSAGE']

        in_ipscore_hi = x['unique_to_ipscore_hi']
        in_hipsci = x['unique_to_hipsci']
        in_ipscore = x['unique_to_ipscore']

        combined_nref_gt = x['NNONREF_GT_COMBINED']
        combined_nref_frac_gt = x['NNONREF_FRAC_GT_COMBINED']

        combined_nref_d = x['NNONREF_DOSAGE_COMBINED']
        combined_nref_frac_d = x['NNONREF_FRAC_DOSAGE_COMBINED']


        combined_categories = ['NNONREF_GT_COMBINED', 'NNONREF_FRAC_GT_COMBINED','NNONREF_DOSAGE_COMBINED',
                               'NNONREF_FRAC_DOSAGE_COMBINED' ]

        combined_categories_alt_names = ['NNONREF_GT_i2QTL', 'NNONREF_FRAC_GT_i2QTL','NNONREF_DOSAGE_i2QTL',
                               'NNONREF_FRAC_DOSAGE_i2QTL' ]




        if x['NNONREF_GT_{}'.format(subtype)] > 0:
            var_classes = [subtype]
            add = get_to_add(var_classes, categories, x)
            info += add

           


            info.append("{}={}".format('CALLER', 'HipSTR'))

            info_str = ";".join(info)
            data.append([ind,info_str])
    
    info_df = pd.DataFrame(data, columns = ['ID', 'INFO'])
    return info_df

In [ ]:
info_df = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/hipstr_combined_annotations.pkl')

In [390]:
info_df = info_column_merged(info_df)

In [176]:
fn_info_df = private_out + '/info_df_merged.txt'
info_df.to_csv(fn_info_df, sep= '\t', index =False, header=False)

In [185]:
fn_info_df_ipscore = private_out + '/info_df_ipscore.txt'
info_df_ipscore = info_column_hipstr_indiv(info_df, subtype='iPSCORE')
info_df_ipscore.to_csv(fn_info_df_ipscore, sep = '\t', index = False, header = False)

In [232]:
fn_info_df_hipsci = private_out + '/info_df_hipsci.txt'
info_df_hipsci = info_column_hipstr_indiv(info_df, subtype='HipSci')
info_df_hipsci.to_csv(fn_info_df_hipsci, sep = '\t', index = False, header = False)

In [209]:
def subset_samples(sample_info, samples_not_fully_consented_fn):
    nc = [i.rstrip() for i in open(samples_not_fully_consented_fn, 'r')]
    WGS_IDs = sample_info[(sample_info.CELL_TYPE != 'iPSC') & (sample_info.STUDY != 'HLI')].WGS_ID.tolist()
    WGS_IDs = sorted(WGS_IDs)
    filtered_WGS_IDs = [i for i in WGS_IDs if i not in nc]
    return WGS_IDs, filtered_WGS_IDs

In [201]:
samples_not_fully_consented_fn = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/ipscore_sample_info/samples_ipscore_not_consented.txt'

In [210]:
all_wgs_ids, filtered_wgs_ids = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/sample_info_combined/sample_info.pkl').pipe(subset_samples, samples_not_fully_consented_fn)

In [351]:
sample_info = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/sample_info_combined/sample_info.pkl')

In [211]:
fn_samples_477 = private_out + '/WGS_Samples_477.txt'
fn_samples_472 = private_out + '/WGS_Samples_472.txt'

In [214]:
with open(fn_samples_477, 'w') as F:
    F.write("\n".join(all_wgs_ids) + '\n')

In [213]:
with open(fn_samples_472, 'w') as F:
    F.write("\n".join(filtered_wgs_ids) + '\n')

In [ ]:
 combined_categories_alt_names = ['NNONREF_GT_i2QTL', 'NNONREF_FRAC_GT_i2QTL','NNONREF_DOSAGE_i2QTL',
                           'NNONREF_FRAC_DOSAGE_i2QTL' ]
   

In [125]:
fn_ipscore = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/ipscore_sample_info/twins.tsv -s /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/ipscore_samples_filt.txt'

In [ ]:
number_col = [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0,0,0,0,1,1,1]

type_col = ['Float','Float','Float','Float','Float','Float','Flag', 'Flag', 'Flag', 'String', 'String', 'String', 'Flag', 'Flag', 'Flag', 'Flag', 'String', 'String', 'String']

In [341]:
description_dict = {'NNONREF_GT_i2QTL': 'Number of samples with non-reference genotype across passing i2QTL samples (272 iPSCORE + 202 HipSci) (non-iPSC)', 
                    'NNONREF_GT_iPSCORE_HipSci_Ref': 'Number of samples with non-reference genotype iPSCORE samples genotyped at HipSci alleles', 
                   'NNONREF_GT_iPSCORE': 'Number of samples with non-reference genotype in iPSCORE samples',
                   'NNONREF_GT_HipSci': 'Number of samples with non-reference genotype in HipSci fibroblast samples',
                    'NNONREF_DOSAGE_i2QTL': 'Number of samples with non-zero change in length from reference (dosage) across passing i2QTL samples (272 iPSCORE + 202 HipSci) (non-iPSC)',
                   'NNONREF_DOSAGE_iPSCORE_HipSci_Ref': 'Number of samples with non-zero change in length from reference (dosage) in iPSCORE samples genotyped at HipSci alleles', 
                   'NNONREF_DOSAGE_iPSCORE': 'Number of samples with non-zero change in length from reference (dosage) in iPSCORE samples',
                   'NNONREF_DOSAGE_HipSci': 'Number of samples with non-zero change in length from reference (dosage) in HipSci fibroblast samples',
                    'NNONREF_FRAC_GT_i2QTL': 'Fraction of samples with non-reference genotype across passing i2QTL samples (272 iPSCORE + 202 HipSci) (non-iPSC)',
                   'NNONREF_FRAC_GT_iPSCORE_HipSci_Ref': 'Fraction of samples with non-reference genotype iPSCORE samples genotyped at HipSci alleles', 
                   'NNONREF_FRAC_GT_iPSCORE': 'Fraction of samples with non-reference genotype in iPSCORE samples',
                   'NNONREF_FRAC_GT_HipSci': 'Fraction of samples with non-reference genotype in HipSci fibroblast samples',
                    'NNONREF_FRAC_DOSAGE_i2QTL': 'Fraction of samples with non-zero change in length from reference (dosage) across passing i2QTL samples (272 iPSCORE + 202 HipSci) (non-iPSC)',
                   'NNONREF_FRAC_DOSAGE_iPSCORE_HipSci_Ref': 'Fraction of samples with nnon-zero change in length from reference (dosage) in iPSCORE samples genotyped at HipSci alleles', 
                   'NNONREF_FRAC_DOSAGE_iPSCORE': 'Fraction of samples with non-zero change in length from reference (dosage) in iPSCORE samples',
                   'NNONREF_FRAC_DOSAGE_HipSci': 'Fraction of samples with non-zero change in length from reference (dosage) in HipSci fibroblast samples',
                    
                    'NNONREF_DOSAGE_i2QTL': 'Number of samples with non-zero change in length from reference (dosage) across passing i2QTL samples (272 iPSCORE + 202 HipSci) (non-iPSC)',
                    'CALLER': 'Variant caller that genotyped this variant (LUMPY/GS/MELT/HipSTR)'}


In [343]:
info_tags = sorted(description_dict.keys())

In [344]:
number_col = [1 for i in info_tags]
type_col =['String'] + ['Float' if i.find('FRAC') != -1 else 'Integer' for i in info_tags[1:]]

In [345]:
data = zip(*[info_tags, number_col, type_col])

In [346]:
header_info_file = pd.DataFrame(data, columns=['ID_COL', 'NUMBER','TYPE'])
header_info_file.index = header_info_file.ID_COL
t = pd.Series(description_dict)
header_info_file['DESCRIPTION'] = t

In [348]:
fn_header_info = private_out + '/header_info.tsv'
header_info_file.to_csv(fn_header_info, sep='\t', index=False)

In [164]:
annotate_all = pd.DataFrame([['CALLER', 1, 'String', 'Variant caller that genotyped this variant (LUMPY/GS/MELT/GS_LCNV/HipSTR)', 'CALLER=HipSTR']], columns= ['ID_COL', 'NUMBER', 'TYPE', 'DESCRIPTION', 'ANNOTATION'])

In [165]:
fn_annotate_all = private_out + '/header_annot_all.tsv'
annotate_all.to_csv(fn_annotate_all, sep = '\t', index=False)

In [ ]:
samples_vcf = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/gs_final_vcf/samples_vcf.pkl')

In [ ]:
samples_consented_fn = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/ipscore_sample_info/samples_ipscore_consented.txt'


In [220]:
fn_samples_477 = private_out + '/WGS_Samples_477.txt'

fn_samples_472 = private_out + '/WGS_Samples_472.txt'

In [224]:
fn_samples = fn_samples_472
vcf_in = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/merged_vcf/hipstr_i2QTL_merged.filt.vcf'
fn_info_df = private_out + '/info_df_merged.txt'
fn_header_info = private_out + '/header_info.tsv'
output_vcf = private_out + '/hipstr_i2QTL_merged.filt.annot.472.vcf'

command = 'python {} -vcf {} -af {} -hf {} -s {} -o {}'.format(annotate_vcf_script, vcf_in, fn_info_df, fn_header_info, fn_samples_472, output_vcf)

print command

python /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/scripts/annotate_vcf_multi_info.py -vcf /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/merged_vcf/hipstr_i2QTL_merged.filt.vcf -af /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/info_df_merged.txt -hf /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/header_info.tsv -s /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/WGS_Samples_472.txt -o /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/hipstr_i2QTL_merged.filt.annot.472.vcf


In [242]:
fn_samples = fn_samples_477
vcf_in = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/merged_vcf/hipstr_i2QTL_merged.filt.vcf'
fn_info_df = private_out + '/info_df_merged.txt'
fn_header_info = private_out + '/header_info.tsv'
output_vcf = private_out + '/hipstr_i2QTL_merged.filt.annot.477.vcf'

command = 'python {} -vcf {} -af {} -hf {} -s {} -o {}'.format(annotate_vcf_script, vcf_in, fn_info_df, fn_header_info, fn_samples_477, output_vcf)
print command

python /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/scripts/annotate_vcf_multi_info.py -vcf /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/merged_vcf/hipstr_i2QTL_merged.filt.vcf -af /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/info_df_merged.txt -hf /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/header_info.tsv -s /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/WGS_Samples_477.txt -o /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/hipstr_i2QTL_merged.filt.annot.477.vcf


In [325]:
fn_samples_ipscore ='/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/sample_info_combined/samples_ipscore.txt'
vcf_in = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/ipscore_all/hipstr_output_annot.vcf'
fn_info_df_ipscore = private_out + '/info_df_ipscore.txt'

fn_header_info = private_out + '/header_info.tsv'
output_vcf = private_out + '/hipstr_iPSCORE.filt.annot.273.vcf'

command = 'python {} -vcf {} -af {} -hf {} -s {} -o {}'.format(annotate_vcf_script, vcf_in, fn_info_df_ipscore,
                                                               fn_header_info, fn_samples_ipscore,
                                                               output_vcf)
print command


python /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/scripts/annotate_vcf_multi_info.py -vcf /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/ipscore_all/hipstr_output_annot.vcf -af /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/info_df_ipscore.txt -hf /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/header_info.tsv -s /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/sample_info_combined/samples_ipscore.txt -o /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/hipstr_iPSCORE.filt.annot.273.vcf


In [362]:
fn_ipscore_samples_consented = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/ipscore_sample_info/samples_ipscore_consented.txt'

vcf_in = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/ipscore_all/hipstr_output_annot.vcf'

fn_info_df_ipscore = private_out + '/info_df_ipscore.txt'

fn_header_info = private_out + '/header_info.tsv'
output_vcf = private_out + '/hipstr_iPSCORE.filt.annot.268.vcf'

command = 'python {} -vcf {} -af {} -hf {} -s {} -o {}'.format(annotate_vcf_script, vcf_in, fn_info_df_ipscore,
                                                               fn_header_info, fn_ipscore_samples_consented,
                                                               output_vcf)
print command

python /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/scripts/annotate_vcf_multi_info.py -vcf /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/ipscore_all/hipstr_output_annot.vcf -af /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/info_df_ipscore.txt -hf /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/header_info.tsv -s /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/ipscore_sample_info/samples_ipscore_consented.txt -o /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/hipstr_iPSCORE.filt.annot.268.vcf


In [361]:
fn_samples = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/sample_info_combined/samples_hipsci_fib.txt'

vcf_in = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/hipsci_filt_geno/hipstr_output_annot.vcf'
fn_info_df_hipsci = private_out + '/info_df_hipsci.txt'

fn_header_info = private_out + '/header_info.tsv'
output_vcf = private_out + '/hipstr_hispci.filt.annot.204.vcf'

command = 'python {} -vcf {} -af {} -hf {} -s {} -o {}'.format(annotate_vcf_script, vcf_in, fn_info_df, fn_header_info,fn_samples , output_vcf)
print command

python /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/scripts/annotate_vcf_multi_info.py -vcf /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_qc_analysis/hipsci_filt_geno/hipstr_output_annot.vcf -af /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/info_df_merged.txt -hf /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/header_info.tsv -s /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/sample_info_combined/samples_hipsci_fib.txt -o /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/hipstr_hispci.filt.annot.204.vcf


# sort and bgzip them 

In [ ]:
%%writefile sortbgzip_hipstr.sh
vcf-sort hipstr_i2QTL_merged.filt.annot.472.vcf | bgzip -c > hipstr_i2QTL_merged.filt.annot.sorted.472.vcf.gz  
vcf-sort hipstr_i2QTL_merged.filt.annot.477.vcf | bgzip -c > hipstr_i2QTL_merged.filt.annot.sorted.477.vcf.gz  
vcf-sort  hipstr_iPSCORE.filt.annot.268.vcf | bgzip -c > hipstr_iPSCORE.filt.annot.sorted.268.vcf.gz  
vcf-sort hipstr_iPSCORE.filt.annot.273.vcf | bgzip -c > hipstr_iPSCORE.filt.annot.sorted.273.vcf.gz  
vcf-sort  hipstr_hispci.filt.annot.204.vcf  | bgzip -c > hipstr_hispci.filt.annot.sorted.204.vcf.gz
vcf-sort hipstr_i2QTL_merged.filt.annot.maf1.ds.rna.415.vcf| bgzip -c > hipstr_i2QTL_merged.filt.annot.maf1.ds.rna.sorted.415.vcf.gz

# Annotate on Dosages and Extract a Dosage VCF for QTL

In [19]:
adjusted_dosage_script = DJ.convert_ipynb_to_script('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/HipSTR_processing/hipstr_annotate_adjusted_dosage_vcf.ipynb')

In [17]:
out_dir = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf'
fn_has_rna='/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/sample_info_combined/samples_i2QTL_rna.txt'
vcf_file = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/hipstr_i2QTL_merged.filt.annot.477.vcf'
name_file_out = 'hipstr_i2QTL_merged.filt.annot.maf.dsf.477.vcf'
command = 'python {} -vcf {} -o {} -s {} -fn_out {}'.format(adjusted_dosage_script, vcf_file, out_dir, fn_has_rna, name_file_out) 

In [14]:
print command

python /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/HipSTR_processing/hipstr_annotate_adjusted_dosage_vcf.py -vcf /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/hipstr_i2QTL_merged.filt.annot.477.vcf -o /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf -s /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/sample_info_combined/samples_i2QTL_rna.txt -fn_out hipstr_i2QTL_merged.filt.annot.maf.dsf.477.vcf


In [20]:
filter_to_dosage_only_script = DJ.convert_ipynb_to_script('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/HipSTR_processing/hipstr_generate_dosage_only_vcf.ipynb')

In [21]:
out_dir = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf'

fn_has_rna='/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/sample_info_combined/samples_i2QTL_rna.txt'
vcf_file = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/hipstr_i2QTL_merged.filt.annot.maf.dsf.477.vcf'
name_file_out = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/hipstr_i2QTL_merged.filt.annot.maf1.ds.rna.415.vcf'


command = 'python {} -vcf {} -o {} -s {} -fn_out {}'.format(filter_to_dosage_only_script, vcf_file, out_dir, fn_has_rna, name_file_out) 

print command

python /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/notebooks/HipSTR_processing/hipstr_generate_dosage_only_vcf.py -vcf /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/hipstr_i2QTL_merged.filt.annot.maf.dsf.477.vcf -o /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf -s /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/sample_info_combined/samples_i2QTL_rna.txt -fn_out /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/hipstr_final_vcf/hipstr_i2QTL_merged.filt.annot.maf1.ds.rna.415.vcf


In [ ]:
# sort and bgzip before combining with MELT/GS/LUMPY
vcf-sort hipstr_i2QTL_merged.filt.annot.maf1.ds.rna.415.vcf | bgzip -c  > hipstr_i2QTL_merged.filt.annot.maf1.ds.rna.sorted.415.vcf.gz
tabix -p vcf hipstr_i2QTL_merged.filt.annot.maf1.ds.rna.sorted.415.vcf.gz